# Making Sense of Data with Pandas

Pandas stands for 'Python Data Analysis Library' and is a package designed to provide data scientists working in Python with a set of powerful tools to load, transform, and process large-ish data sets. As a result, it has become something of a *de facto* standard and many of the tutorials that you find online will probably make use of pandas at some point. It is provided by default with Anaconda Python.

You will probably want to bookmark [the documentation](http://pandas.pydata.org/pandas-docs/stable/) since you will undoubtedly need to refer to it fairly regularly. _Note_: this link is to the most recent, stable release. If you are using an older version of pandas then you'll need to track it down from the [home page](http://pandas.pydata.org).

You can always check what version you have installed like this:
```python
import pandas as pd
print pd.__version__
```
This won't work with _all_ packages, but it will work with a lot of them. Remember that variables and methods starting and ending with '`__`' are **private**. So you shouldn't try to change this or manipulate it unless you really, really know what you are doing. Actually, just don't ever do it unless you're contributing to the package's development.

Anyway, the main elements of pandas with which you interact directly are: 1) the DataFrame; 2) the Series. Let's take a look at one of them:

In [ ]:
import pandas as pd
help(pd.DataFrame)

On second thought, let's never do that again. Well, at least not _that_ way! You'll have noticed that the help documentation for the DataFrame is not just a bit longer than anything we've seen before, it's massively longer. There's probably quite a lot of intimidating terminology in there too... Right from the start we get things like "Two-dimensional size-mutable, potentially heterogeneous tabular data structure with labeled axes (rows and columns)." 

Here's the thing: in the [last notebook](https://raw.githubusercontent.com/kingsgeocomp/geocomputation/master/Practical-4-Functions%2C%20Packages%20and%20Methods.ipynb) we came close to inventing something a lot like pandas from scratch. 

So you already _know_ what's going on, or at least have an analogy that you can use to make sense of it. Pandas take a column-view of data in the same way that our Dictionary-of-Lists did, it's just got a lot more features to help you work with the data than our DoL does. That's why the documentation is so much more forbidding. And why pandas is so much more powerful.

But at its heart, a pandas data frame ('df' for short) is just a collection of data series (i.e. columns) with an index. Each Series is like one of our column-lists from the last notebook. And the DataFrame is like the overarching dictionary that held the collection of data series (serieses?) together. OK? You've seen this before.

Let's try it with last week's data!

In [ ]:
df = pd.read_csv('http://www.reades.com/CitiesWithWikipediaData.csv')

df.head()

Check it out!

Instead of having to write a 'readRemoteCSV' function and then manually create a Dictionary-of-Lists from that remote file, we just told pandas to read it for us and it automagically converted it to a data structure that we could view. You'll notice that it even figured out where the column names were. 

All we head with `df.head()` was ask it to print out the first 5 rows of data. If we wanted to only see the first two rows it would be `df.head(2)`. This is pretty handy, right? 

Let's try a few more things:

In [ ]:
df.describe()

You'll probably have seen a fairly prominent warning ("Invalid value encountered in percentile"), and if you look closely you'll see that there are some fields that report things like '`NaN`' in some of the rows. These are related, but let's take a step back for a second: by calling the `describe` we were able to produce a 7-figure summary for _most_ of the columns in the data! That's a pretty handy way to summarise what's in there, right?

So, just be calling `describe`...
1. We've asked Python to describe the data frame and it has returned a set of columns with descriptive metrics for each.
2. Note what is _missing_ from this list: where are 'Name', 'MetroArea', and a couple of the other columns? Can you think why they weren't reported in the descriptives?
3. For the other columns, notice that `NaN`. `NaN` is short-hand for 'Not a Number' an it flags up a potential problem when we are dealing with numeric columns because it's hard to know what do with something like that in a column that should contain only numbers. Is something that isn't a number something that should be ignored? Is it a major problem? Is it a 'we don't know the value' or 'we couldn't read the value'? Those are different problems!

Of course, maybe you don't want all of this, maybe you're just interested in one column:

In [ ]:
print df.Population.describe()

So now we have the same information, but only for the Population column. We have to do this a _little_ differently because describing the DataFrame does some clever formatting, but describe a Series requires us to print out the result. Also notice that `dtype` at the end: that tells us the _data type_ is a 64-bit float. You can have strings, floats, integers, booleans, etc. in a DataFrame.

But the really crucial thing is that this introduces _one_ of the two ways that we access a Series in pandas: `<data frame>.<series name>.method`. So we could get similar information on the Name column with:
```python
df.Name.describe()
```
And so forth.

In [ ]:
print df.Name.describe()
print " "
print "The mean is: " + str(df.Population.mean())

Notice that describing a text column gives us an 'object' data type because a String is a complex object, not a simple float or int.

And notice to that we can ask for the a derived variable (such as the mean) just be asking the Series to do the work for us: `<data frame>.<series>.method()`. You might want to have a [look at the documentation](http://pandas.pydata.org/pandas-docs/stable/api.html#series) to see what other methods are available for a data series. It's rather a long list.

### A Challenge for You!

If all of this has made some kind of sense, why not spend a few minutes exploring the data using pandas. Why do you think that there are a large number of "Unnamed" columns with NaNs in them? Try looking at the data to see if you can figure it out... here's a clue: the equivalent of `myList[5]` in a DataFrame is...
```python
df.iloc[5]
```
Use the coding block below for your exploration.

OK, now that we've seen the _bare bones basics_ of pandas, let's start working with some real data.

# Working with Data

One of the key things to do when working with any new data set is to familiarise yourself with it. You can do this in a variety of ways:
* Read sources about the data, e.g. ONS (2010)
* Review any accompanying metadata (data about the data)
* Looking at the data itself, including comparing it to a) and b)
* Make initial data analyses, including descriptive statistics 
* Visualize the data using plots 
In fact, you should use _all_ of these together to really understand where the data came from, how it was handled, and whether there are gaps or other problems.

For the remainder of this module we're going to be working with two types of data: data about people (Socio-economic Classifcation) and data about the environment (weather). We've selected two very different types of data on purpose:
1. Because we know that some of you have interests in the human environment, and others in the natural
2. Because these are very different types of data with very different properties
3. Because we'll see that _similar_ workflows can be used with each!
What we want to highlight is that computational approaches are _highly transferrable_ between contexts. The mean or median is not _less_ relevant in one context than another, it's just more or less appropriate as a tool for understanding the data! 

### A Bit of Context

The statistician George Box [once said](https://en.wikipedia.org/wiki/All_models_are_wrong) "all models are wrong but some are useful". Now you might think that 'wrong' _implies_ uselessness, but this aphorism is a lot more interesting than that: if you're following this course in-person, then you'll have come across the idea of statistics as the study of a 'data generating process'. 

The data that we work with is a _representation_ of reality, it is not reality itself. Just because I can say that the height of human beings is normally distributed with a mean of, say, 170cm and standard deviation of 8cm doesn't mean that I've _explained_ that process. What I have done is to say that reality can be reasonably well approximated by a normal distribution with those features. 

Given that understanding, I know that someone with a heigh of 2m is very, very improbable. Not impossible, just highly unlikely. So if I meet someone that tall then that's quite exciting! But my use of a normal distribution to represent that reality doesn't mean that I think that height is randomly distributed amongst all human beings like some gigantic lottery system: some parts of the world are typically shorter, other parts typcially taller. 

So the _real_ reason for someone's height is to do with, I assume, genetics and nutrition, but across large groups of people it's possible to _represent_ the cumulative impact of those individual realities with a simple normal distribution. And using that simplified data generating process allows me to do things like estimate the likelihood of meeting someone 2m tall (which is why I'd be excited to do so).

In the same way, real individuals earn different incomes for all sorts of reasons: skills, education, negotiation ability... and systematic discrimination or bias. Because of wide variations in individual lived experience, it's quite hard to _prove_ that any _one_ person has been discriminated against unless you have the 'smoking gun' of an email or other direct evidence. 

But if I have data on _many_ men and women (from either inside or outside of the company) to work with, then I can take a look at what data generting processes best describe what I've observed. And I can also create a data generating process that would describe what I'd _expect_ if there were no discrimination taking place. 

And if I find a significant difference then I can start to rule out claims that 'there are no good candidates' and the other redoubts of the indefensible. It is _possible_ that a company had trouble finding qualified candidates, but as you put together the evidence using statistics it becomes increasingly _improbable_.

So, always remember that the data is not reality. But it is a very useful abstraction of reality that allows us to search for larger patterns and to make certain claims about the observations in the data based on what we know about the characteristics of the data generating process.

Here's a (slightly terrifying) video to explain this another way:

[![From reality to make-believe](http://img.youtube.com/vi/HAfI0g_S9oo/0.jpg)](https://www.youtube.com/watch?v=HAfI0g_S9oo)

### Socio-economic Classification Data

Many governments publish large data sets drawn from a national Census that takes place every 5 or 10 years. And we know that things like education and income affect people's life chances in the long-run: higher levels of education, higher incomes, more senior jobs, and owning a house (instead of renting) tend to be linked to better health and other long-term advantages. 

So if we want to understand the overall distribution of these advantages in the population as a whole it might be handy to derive a 'class' for every household based on their levels of education, their job, their, health, and so forth. We can then look at how those classes are distributed across an area to see if there is mixing (generally thought to be a good thing) or to select an area for further investigation.

In the UK this [classifcation](http://www.statistics.gov.uk/downloads/theme_compendia/ESRC_Review.pdf) is known as the National Statistics Socio-economic Classifcation (NS-SeC). You really should [read the documentation](http://webarchive.nationalarchives.gov.uk/20160105160709/http://www.ons.gov.uk/ons/guide-method/classifications/current-standard-classifications/soc2010/soc2010-volume-3-ns-sec--rebased-on-soc2010--user-manual/index.html) before trying to work with this data for research (or an end-of-term essay) but the main point is the 8-fold classifcation:

| Analytic Class | Description |
| --- | --- |
| 1 | Higher managerial and professional occupations |
| 2 | Lower managerial and professional occupations |
| 3 | Intermediate occupations (clerical, sales, service) |
| 4 | Small employers and own account workers |
| 5 | Lower supervisory and technical occupations |
| 6 | Semi-routine occupations |
| 7 | Routine occupations |
| 8 | Never worked or long-term unemployed |
| NC | Not classified elsewhere |

These can be grouped into a 3-fold classification:
| Class | Description |
| --- | --- |
| 1 | Higher occupations |
| 2 | Intermediate occupations |
| 3 | Lower occupations |

And they can also be disaggregated into more detailed groups a limited way.

### Weather Data 

I've got a lot less to say about weather data at the moment because you've probably been making use of forecasts for much of your life! But it's still worth understanding something about how weather data is gathered and reported: many organisations operate weather stations where data on wind speed, temperature, rain, and amount of sun are collected and then transmitted to a server to be integrated into a larger data set of weather _observations_ at a national or global scale. Of course, any _one_ station might be in the 'wrong' place (somewhere shady or protected from the rain) or it might even break down, but the idea is that if you have enough of them you can collect a pretty good range of data for the country and begin to look for patterns and, potentially, make predictions.

----
# Getting the NS-SeC Data

## Using InFuse
To obtain the NS-SeC data you'll have to visit the [InFuse](http://infuse.mimas.ac.uk) web site. Here's what you'll need to do:
1. Select the 2011 Census data
2. Scroll down the 'Filters' on the left-hand side of the page until you find 'NS-Sec' (*note*: there are _four_ options)
3. Select the first NS-SeC filter: "NS-SeC (National Statistics Socio-economic Classification) of household reference person"
4. Select the first option (Usual resident population; NS-SeC) from the range of data sets offered
5. Read about the selected topic and then click 'Next'
6. Select 'Total', classes 1–8, and Not classified using the checkboxes.
7. Click 'Add' to add these to the 'Selected category combinations' below (it should say "You have selected 10 category combinations)
8. Click 'Next'
9. Select 'Middle Super Output Areas' (8,480 areas) from the checkboxes and then click 'Add'
10. Click 'Next', review the data extract information, and then click 'Get the data'
11. When the big red 'Download the data' button appears after a few seconds click 'Download'
12. Save the data to the _same folder_ as this notebook.

If you've followed the steps above carefully, you should now have a Zip file showing in the 'Home' tab of your browser (the tab from which you launched this notebook).

## Processing the data

You could always unzip the downloaded file manually by double-clicking the zipfile, but given that we're programming in Python perhaps we could do it a different way?

Perhaps [Google](http://www.google.com/) has the answer to 'unzipping zip files python'? And perhaps the first answer in that list will be something from [StackOverflow](http://stackoverflow.com/questions/3451111/unzipping-files-in-python)? 

In [ ]:
import zipfile
zip_ref = ???
zip_ref.extractall('./Data/')
zip_ref.close()

If all has gone well you should now have a folder called 'Data' inside the folder where this notebook sits. The Data folder contains three files:
1. citations.rtf – this is information about how to reference this data set so that other people know where it came from. 
2. Data_NSSHRP_UNIT_URESPOP.csv – this looks promising...
3. Meta_NSSHRP_UNIT_URESPOP.csv – this file takes a little getting used to but it's very, very important since it's the metadata that describes each column in our downloaded data file!

The hardest thing about the metadata file is that several rows can relate to _one_ column in the data file. That's because each row is giving us _different_ information about what the column contains: what's it about, where did it come from, what format is it in? Etc.

Let's be _bit_ naughty and just see what happens if we try to load the data file directly into pandas...

In [3]:
import pandas as pd
df = pd.read_csv('./Data/Data_NSSHRP_UNIT_URESPOP.csv')

df.head(3)

,CDU_ID,GEO_CODE,GEO_LABEL,GEO_TYPE,GEO_TYP2,F2084,F2085,F2094,F2102,F2107,F2114,F2119,F2127,F2133,F2136,Unnamed: 15
0,NaN,NaN,NaN,NaN,NaN,NS-SeC (National Statistics Socio-economic Cla...,NS-SeC (National Statistics Socio-economic Cla...,NS-SeC (National Statistics Socio-economic Cla...,NS-SeC (National Statistics Socio-economic Cla...,NS-SeC (National Statistics Socio-economic Cla...,NS-SeC (National Statistics Socio-economic Cla...,NS-SeC (National Statistics Socio-economic Cla...,NS-SeC (National Statistics Socio-economic Cla...,NS-SeC (National Statistics Socio-economic Cla...,NS-SeC (National Statistics Socio-economic Cla...,NaN
1,9937.0,E02000001,City of London 001,Middle Super Output Areas and Intermediate Zones,MSOAIZ,7187,2730,2246,543,497,224,308,212,178,249,NaN
2,9938.0,E02000002,Barking and Dagenham 001,Middle Super Output Areas and Intermediate Zones,MSOAIZ,6724,340,1180,785,871,526,1276,963,589,194,NaN


It should be pretty obvious that this isn't quite what we want: although we did get the variable names from the first row of the file, row 0 of our data frame still contains metadata! It's time to see what we can do in pandas to tidy this up _when_ we do the import.

I'd suggest looking at the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) for `read_csv`, and in particular at the following options that can be _passed_ to `read_csv()`:
* header
* names
* usecols
* skiprows
* nrows

By way of guidance:
* I would _suggest_ that you skip the first couple of rows
* I would _suggest_ that you drop (i.e. ignore) some of the columns 
* I would _suggest_ that you specify your own column names
* I would _suggest_ that while you get this right you only read a small number of rows

Answer at the end of this notebook, but try it yourself first by adding one parameter at a time to _build_ a working import statement (don't try all of these options at once!). But by way of a hint to get you started and to ensure that you end up with the same column names that I do:
```python
colnames = ['CDU','GeoCode','GeoLabel','GeoType','GeoType2','Total']
for i in range(1,8):
    colnames.append('Group' + str(i))
colnames.append('NC')
```
Now over to you!

You've now written the components of a Python script that can take a file downloaded from InFuse, automatically extract the contents from a Zip archive, and then load the data into pandas automatically. Now that we've done it for _one_ file, we can work out how to do it for _any_ file. That's what we mean by scalability: yes, the column names will be different for other files downloaded from InFuse, but the _process_ is the same: we could create a function that handles all of this for us and the only thing it would need is the names that we want to use for the columns! 

----
# Getting Weather Data

Now let's switch gear a bit. The UK's Met Office is a world-leading weather and climate research centre, and even if it doesn't always seem like their forecasts are very accurate that's because Britain's weather is inherently _unpredictable_. They've also done a lot of work to make their weather data widely available to people like us.

But because the weather is changing all the time, so is the data! And, 'worse', it's becoming obsolete: the forecast from 2 years ago isn't particularly useful to us now. And asking for 'yesterday's weather' depends on the day that we're asking! When you have data in something like 'real-time' you don't normally access it the same way: you use something called an API (Application Programming Interface) that is designed with programmatic interaction in mind right from the start.

Helpfully, the MetOffice provides a lot of documentation about their API (I'd suggest bookmarking it): http://www.metoffice.gov.uk/datapoint/support/api-reference

This type of data requires a lot more research up front to work with, but it's very flexible once you know how it works because you can _customise_ the extract to obtain only the fields that you need instead of being 'stuck' with what the provider wants to give you.

## Obtaining an API Key

The first step to working with the API from the MetOffice is to obtain an API key: [do that here](http://www.metoffice.gov.uk/datapoint/API).

## Making an API Request

We then use the key as part of an API request: the process by which we _ask_ for data. We're going to show you the code and output first and then we'll talk through the steps involved. But, first, you'll need to replace "???" with the API key provided to you by the MetOffice.

In [28]:
import json, requests

api_key   = "???" # your API key
api_url   = "http://datapoint.metoffice.gov.uk/public/data/" # base URL
obs_json  = "val/wxobs/all/json/" # observations URL
fcs_json  = "val/wxfcs/all/json/" # forecasts URL

heathrow = str(3772)  # heathrow airport weather station

payload = {'res': 'hourly', 'key': api_key}
r = requests.get(api_url + obs_json + heathrow, params=payload)

#check the call - need some proper try, except stuff here
print(r.url)

#check the output
print(r.json())

http://datapoint.metoffice.gov.uk/public/data/val/wxobs/all/json/3772?res=hourly&key=f997090f-8b2a-444e-8ceb-260f07094eb1
{u'SiteRep': {u'DV': {u'type': u'Obs', u'dataDate': u'2016-09-09T12:00:00Z', u'Location': {u'elevation': u'25.0', u'name': u'HEATHROW', u'i': u'3772', u'country': u'ENGLAND', u'lon': u'-0.4491', u'Period': [{u'Rep': [{u'D': u'WSW', u'Pt': u'R', u'H': u'44.2', u'P': u'1011', u'S': u'19', u'T': u'23.2', u'W': u'1', u'V': u'30000', u'Dp': u'10.4', u'$': u'720'}, {u'D': u'WSW', u'Pt': u'R', u'H': u'43.3', u'P': u'1012', u'S': u'18', u'T': u'23.3', u'W': u'1', u'V': u'15000', u'Dp': u'10.2', u'$': u'780'}, {u'D': u'W', u'Pt': u'R', u'H': u'45.2', u'P': u'1012', u'S': u'16', u'T': u'22.8', u'W': u'7', u'V': u'30000', u'Dp': u'10.4', u'$': u'840'}, {u'D': u'W', u'G': u'29', u'Dp': u'10.5', u'H': u'46.0', u'P': u'1012', u'S': u'16', u'T': u'22.6', u'W': u'7', u'V': u'30000', u'Pt': u'R', u'$': u'900'}, {u'D': u'W', u'Pt': u'R', u'H': u'46.4', u'P': u'1012', u'S': u'14', u'T

OK, now let's make sense of this:
```python
import json, requests

api_key   = "???" # your API key
api_url   = "http://datapoint.metoffice.gov.uk/public/data/" # base URL
obs_json  = "val/wxobs/all/json/" # observations URL
fcs_json  = "val/wxfcs/all/json/" # forecasts URL
```
So, first we import two new modules: one that makes requests to a web server, and one that will parse JSON responses from the server in order to turn them into something that we can use.

Then we set up some default values that will allow us to build our request to the MetOffice server. The comments help us to remember what each of these variables is.

Now let's do the actual work:
```python
heathrow = str(3772)  # heathrow airport weather station

payload = {'res': 'hourly', 'key': api_key}
r = requests.get(api_url + obs_json + heathrow, params=payload)

# Check the call - need some proper try, except stuff here
print(r.url)

# Check the output
print(r.json())
```
We want the data for Heathrow Airport: we have to request it using a unique identifier (3772 in this case) because that's easier for the computer to handle than a long, potentially ambiguous string. For instance, if you asked for 'London' what would you get? The City of London? Greater London? 

We can then assemble a URL request by combining the site name, the observations URL, and the parameters. In this case that's: the type of 'resource' (the hourly observations), and our API key.

The last two steps are just about printing out the reply... It's pretty hard to figure out what that reply means, but it's actually just a kind of dictionary. That's it. It looks like a mess, but it _is_ a dictionary and the only thing that is entirely new is the fact that every string has the letter 'u' in front of it. That 'u' means 'Unicode' and it just a kind of string...

It might be a little easier to read if we just look at the description.

In [30]:
pdesc = r.json()['SiteRep']['Wx']
pdat  = r.json()['SiteRep']['DV']

print(pdesc)

{u'Param': [{u'units': u'mph', u'name': u'G', u'$': u'Wind Gust'}, {u'units': u'C', u'name': u'T', u'$': u'Temperature'}, {u'units': u'm', u'name': u'V', u'$': u'Visibility'}, {u'units': u'compass', u'name': u'D', u'$': u'Wind Direction'}, {u'units': u'mph', u'name': u'S', u'$': u'Wind Speed'}, {u'units': u'', u'name': u'W', u'$': u'Weather Type'}, {u'units': u'hpa', u'name': u'P', u'$': u'Pressure'}, {u'units': u'Pa/s', u'name': u'Pt', u'$': u'Pressure Tendency'}, {u'units': u'C', u'name': u'Dp', u'$': u'Dew Point'}, {u'units': u'%', u'name': u'H', u'$': u'Screen Relative Humidity'}]}


Notice how the above also looks a lot like a mix of Python dictionaries and lists: '{' and '['.

Let's reformat it a bit using _recursion_:

In [49]:
def outputList(l, depth): 
    for i in range(len(l)):
        value = l[i]
        if type(value) is list:
            outputList(value, depth+1)
        elif type(value) is dict: 
            outputDict(value, depth+1)
        else:
            print "\t" * depth + "lValue: " + value
    print "\n"

def outputDict(d, depth):
    for key, value in d.iteritems():
        print "\t" * depth + "dKey: " + key
        if type(value) is list:
            outputList(value, depth+1)
        elif type(value) is dict:
            outputDict(value, depth+1)
        else:
            print "\t" * depth + "dValue: " + value
    print "\n"

outputDict(pdesc, 0)

dKey: Param
		dKey: units
		dValue: mph
		dKey: name
		dValue: G
		dKey: $
		dValue: Wind Gust


		dKey: units
		dValue: C
		dKey: name
		dValue: T
		dKey: $
		dValue: Temperature


		dKey: units
		dValue: m
		dKey: name
		dValue: V
		dKey: $
		dValue: Visibility


		dKey: units
		dValue: compass
		dKey: name
		dValue: D
		dKey: $
		dValue: Wind Direction


		dKey: units
		dValue: mph
		dKey: name
		dValue: S
		dKey: $
		dValue: Wind Speed


		dKey: units
		dValue: 
		dKey: name
		dValue: W
		dKey: $
		dValue: Weather Type


		dKey: units
		dValue: hpa
		dKey: name
		dValue: P
		dKey: $
		dValue: Pressure


		dKey: units
		dValue: Pa/s
		dKey: name
		dValue: Pt
		dKey: $
		dValue: Pressure Tendency


		dKey: units
		dValue: C
		dKey: name
		dValue: Dp
		dKey: $
		dValue: Dew Point


		dKey: units
		dValue: %
		dKey: name
		dValue: H
		dKey: $
		dValue: Screen Relative Humidity








I wouldn't get too hung up on how recursion works, but it's a _very_ powerful concept: in this case we know that we're starting with a dictionary so we ask `outputDict` to start outputting the content of `pdesc` (the rePly DESCription). `outputDict` then takes each of the key/value pairs in turn, looks at the value to see if _it_ is a dictionary or list or (by default) string and takes appropriate action. We've also got an `outputList` function that does the same thing: it looks at the elements of the list and either prints out the item or passes it back to a function to process further. As I said, don't get too stressed out if it doesn't make sense just yet, but it's worth gradually getting to grips with it.

We can do the same with the data in the reply:

In [50]:
outputDict(pdat, 0)

dKey: type
dValue: Obs
dKey: dataDate
dValue: 2016-09-09T12:00:00Z
dKey: Location
	dKey: elevation
	dValue: 25.0
	dKey: name
	dValue: HEATHROW
	dKey: i
	dValue: 3772
	dKey: country
	dValue: ENGLAND
	dKey: lon
	dValue: -0.4491
	dKey: Period
			dKey: Rep
					dKey: D
					dValue: WSW
					dKey: Pt
					dValue: R
					dKey: H
					dValue: 44.2
					dKey: P
					dValue: 1011
					dKey: S
					dValue: 19
					dKey: T
					dValue: 23.2
					dKey: W
					dValue: 1
					dKey: V
					dValue: 30000
					dKey: Dp
					dValue: 10.4
					dKey: $
					dValue: 720


					dKey: D
					dValue: WSW
					dKey: Pt
					dValue: R
					dKey: H
					dValue: 43.3
					dKey: P
					dValue: 1012
					dKey: S
					dValue: 18
					dKey: T
					dValue: 23.3
					dKey: W
					dValue: 1
					dKey: V
					dValue: 15000
					dKey: Dp
					dValue: 10.2
					dKey: $
					dValue: 780


					dKey: D
					dValue: W
					dKey: Pt
					dValue: R
					dKey: H
					dValue: 45.2
					dKey: P
					dValue: 1012
					dKey: S
					

In [55]:
import time 

if pdat['type'] != "Obs":
    print("Errrr, these aren't observations!:")

# Ignore the time part as we're getting data with values in minutes
# after midnight so we want to reset this to 00:00:00Z
obsDate = time.strptime(pdat['dataDate'].split("T")[0],'%Y-%m-%d')

print obsDate

time.struct_time(tm_year=2016, tm_mon=9, tm_mday=9, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=4, tm_yday=253, tm_isdst=-1)


In [57]:
# Count number of locations to process
print("There are {0} locations to process...".format(len(pdat['Location'])))

locations = list()
locations.append(['ID','Name','Country','Continent','Latitude','Longitude','Elevation'])

observations = list()
observations.append(['ID','Time','WeatherType','Visibility','Temperature','WindDirection',
    'WindSpeed','WindGust','PressureTendency','DewPoint','Humidity'])

print pdat['Location']

for loc in pdat['Location']:
    # Get the location data and separate it from the 
    # observation data
    locID = int(loc['i'])
    locations.append([locID, loc['name'].title(), loc['country'].title(), loc['continent'].title(), 
        float(loc['lat']), float(loc['lon']), float(loc['elevation']) ])
        
    # Now deal with the actual observations
    for obs in loc['Period']: 
        if obs['type'] != 'Day' and str(obs['value']) != time.strftime("%Y-%m-%dZ", obsDate):
            print("Something has gone a bit wrong: either observation type is not day, or observation value is not the same as the file's")
            break
        for report in obs['Rep']:
            minutes_after_midnight = int(report['$'])
            ts = dt.datetime.fromtimestamp(time.mktime(obsDate)) + dt.timedelta(0, minutes_after_midnight*60)
            
            for key in ['D','Pt']:
                if key not in report:
                    report[key] = u""
            for key in ['W','V','S','G']:
                if key not in report or report[key] == "":
                    report[key] = "0"
            for key in ['T','Dp','H']:
                if key not in report or report[key] == "":
                    report[key] = "0.0"          
            
            observations.append([ locID, str(ts), int(report['W']), int(report['V']), float(report['T']), str(report['D']), 
                int(report['S']), int(report['G']), str(report['Pt']), float(report['Dp']), float(report['H']) ])

print observations

There are 8 locations to process...
{u'elevation': u'25.0', u'name': u'HEATHROW', u'i': u'3772', u'country': u'ENGLAND', u'lon': u'-0.4491', u'Period': [{u'Rep': [{u'D': u'WSW', u'Pt': u'R', u'H': u'44.2', u'P': u'1011', u'S': u'19', u'T': u'23.2', u'W': u'1', u'V': u'30000', u'Dp': u'10.4', u'$': u'720'}, {u'D': u'WSW', u'Pt': u'R', u'H': u'43.3', u'P': u'1012', u'S': u'18', u'T': u'23.3', u'W': u'1', u'V': u'15000', u'Dp': u'10.2', u'$': u'780'}, {u'D': u'W', u'Pt': u'R', u'H': u'45.2', u'P': u'1012', u'S': u'16', u'T': u'22.8', u'W': u'7', u'V': u'30000', u'Dp': u'10.4', u'$': u'840'}, {u'D': u'W', u'G': u'29', u'Dp': u'10.5', u'H': u'46.0', u'P': u'1012', u'S': u'16', u'T': u'22.6', u'W': u'7', u'V': u'30000', u'Pt': u'R', u'$': u'900'}, {u'D': u'W', u'Pt': u'R', u'H': u'46.4', u'P': u'1012', u'S': u'14', u'T': u'22.7', u'W': u'3', u'V': u'40000', u'Dp': u'10.7', u'$': u'960'}, {u'D': u'W', u'Pt': u'R', u'H': u'47.7', u'P': u'1013', u'S': u'15', u'T': u'21.8', u'W': u'1', u'V': u'4

TypeError: string indices must be integers

In [ ]:
# Write CSV of Locations
fn = "-".join(["scope", scope, "obs", str(dt.date.today()), 'locations']) + ".csv"
with open(os.path.join(path, fn), 'wb') as fh:
    writer = csv.writer(fh)
    writer.writerows(locations)

# Write CSV of Observations
fn = "-".join(["scope", scope, "obs", str(dt.date.today()), 'observations']) + ".csv"
with open(os.path.join(path, fn), 'wb') as fh:
    writer = csv.writer(fh)
    writer.writerows(observations)

# Appendix 1: Building a read_csv() Call for Pandas

In [14]:
# =======================
# Building a more complex read_csv call
# to import the NS-SeC Data
# =======================
df = pd.read_csv('./Data/Data_NSSHRP_UNIT_URESPOP.csv', nrows=1)
df.head()

,CDU_ID,GEO_CODE,GEO_LABEL,GEO_TYPE,GEO_TYP2,F2084,F2085,F2094,F2102,F2107,F2114,F2119,F2127,F2133,F2136,Unnamed: 15
0,NaN,NaN,NaN,NaN,NaN,NS-SeC (National Statistics Socio-economic Cla...,NS-SeC (National Statistics Socio-economic Cla...,NS-SeC (National Statistics Socio-economic Cla...,NS-SeC (National Statistics Socio-economic Cla...,NS-SeC (National Statistics Socio-economic Cla...,NS-SeC (National Statistics Socio-economic Cla...,NS-SeC (National Statistics Socio-economic Cla...,NS-SeC (National Statistics Socio-economic Cla...,NS-SeC (National Statistics Socio-economic Cla...,NS-SeC (National Statistics Socio-economic Cla...,NaN


In [51]:
df = pd.read_csv('./Data/Data_NSSHRP_UNIT_URESPOP.csv', nrows=1, skiprows=[1])
df.head()

,CDU_ID,GEO_CODE,GEO_LABEL,GEO_TYPE,GEO_TYP2,F2084,F2085,F2094,F2102,F2107,F2114,F2119,F2127,F2133,F2136,Unnamed: 15
0,9937,E02000001,City of London 001,Middle Super Output Areas and Intermediate Zones,MSOAIZ,7187,2730,2246,543,497,224,308,212,178,249,NaN


In [52]:
colnames = ['CDU','GeoCode','GeoLabel','GeoType','GeoType2','Total']
for i in range(1,8):
    colnames.append('Group' + str(i))
colnames.append('NC')
df = pd.read_csv('./Data/Data_NSSHRP_UNIT_URESPOP.csv', nrows=2, header=0, skiprows=[1], names=colnames)

df.head() # Success!

,,CDU,GeoCode,GeoLabel,GeoType,GeoType2,Total,Group1,Group2,Group3,Group4,Group5,Group6,Group7,NC
9937,E02000001,City of London 001,Middle Super Output Areas and Intermediate Zones,MSOAIZ,7187,2730,2246,543,497,224,308,212,178,249,NaN
9938,E02000002,Barking and Dagenham 001,Middle Super Output Areas and Intermediate Zones,MSOAIZ,6724,340,1180,785,871,526,1276,963,589,194,NaN
